## 2017-2019 Moives Box Office Exploratory Exercise ##

1. We can split data set by MPAA, GENRE
2. Do certain genres make better return(rev-budget), scatter plot of rev vs budget, weighted by genre
3. Does it help to span multiple genres
4. Does the statemant hold true for worldwide. 
5. Each studios slices of the pie

In [33]:
import pandas as pd
import seaborn as sns
import numpy as np

import plotly.express as px
from dash import html, dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

In [34]:
from controls import MPAA, GENRES, GENRES_COLORS
import helper
MPAA_options = [ {"label": str(MPAA[rating]), "value": str(rating)} for rating in MPAA ]
GENRES_options = [ {"label": str(GENRES[genre]), "value": str(genre)} for genre in GENRES ]
GENRES_COLORS_options = [ {"label": str(GENRES[color]), "value": str(color)} for color in GENRES_COLORS ]


plot_layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
)


In [35]:
data = pd.read_csv('data/boxoffice2017_2019.csv', encoding = 'ISO-8859-1')
data['MPAA'].fillna('Not Rated', inplace=True)
data['genres'].fillna('Unknown', inplace=True)
data.head(2)

,title,domestic_revenue,world_revenue,distributor,opening_revenue,opening_theaters,budget,MPAA,genres,release_days
0,Star Wars: Episode VIII - The Last Jedi,"$620,181,382","$1,332,539,889",Walt Disney Studios Motion Pictures,"$220,009,584","4,232","$317,000,000",PG-13,"Action,Adventure,Fantasy,Sci-Fi",382
1,The Fate of the Furious,"$226,008,385","$1,236,005,118",Universal Pictures,"$98,786,705","4,310","$250,000,000",PG-13,"Action,Adventure,Thriller",262


In [36]:
selected_rating =['PG13', 'PG', 'R', 'NA', 'G', 'NC17', 'MPG', 'TVPG']
selected_genres = ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'FilmNoir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport', 'Thriller', 'Unknown', 'War', 'Wester']

In [37]:
BO_dataf = helper.filter_movies_by_ratings(data, selected_rating)
BO_dataf.head(5)

,title,domestic_revenue,world_revenue,distributor,opening_revenue,opening_theaters,budget,MPAA,genres,release_days
4,Beauty and the Beast,"$504,014,165","$1,263,521,126",Walt Disney Studios Motion Pictures,"$174,750,616","4,210","$160,000,000",PG,"Family,Fantasy,Musical,Romance",290
5,It,"$327,481,748","$700,381,748",Warner Bros.,"$123,403,419","4,103","$35,000,000",R,Horror,119
8,Logan,"$226,277,068","$619,021,436",Twentieth Century Fox,"$88,411,916","4,071","$97,000,000",R,"Action,Drama,Sci-Fi,Thriller",304
9,Despicable Me 3,"$264,624,300","$1,034,799,409",Universal Pictures,"$72,434,025","4,529","$80,000,000",PG,"Adventure,Animation,Comedy,Family,Fantasy,Sci-Fi",189
10,Girls Trip,"$115,171,585","$140,552,359",Universal Pictures,"$31,201,920","2,591","$19,000,000",R,"Adventure,Comedy,Drama",164


In [39]:
data_numeric=BO_dataf[['domestic_revenue','world_revenue','opening_revenue','opening_theaters','budget','release_days']].replace('[\$,]', '', regex=True).astype(float)
data_numeric['domestic_revenue_log'] = np.log10(data_numeric['domestic_revenue'])
data_numeric['world_revenue_log'] = np.log10(data_numeric['world_revenue'])
data_numeric['opening_revenue_log'] = np.log10(data_numeric['opening_revenue'])
data_numeric['budget_log'] = np.log10(data_numeric['budget'])
data_numeric['genres'] = BO_dataf['genres']
data_numeric['MPAA'] = BO_dataf['MPAA']
data_numeric['distributor'] = BO_dataf['distributor']
# data_numeric.drop(['domestic_revenue', 'world_revenue','opening_revenue','budget'], axis=1, inplace=True)



In [40]:
data_numeric

,domestic_revenue,world_revenue,opening_revenue,opening_theaters,budget,release_days,domestic_revenue_log,world_revenue_log,opening_revenue_log,budget_log,genres,MPAA,distributor
4,504014165.0,1.263521e+09,174750616.0,4210.0,160000000.0,290.0,8.702443,9.101583,8.242419,8.204120,"Family,Fantasy,Musical,Romance",PG,Walt Disney Studios Motion Pictures
5,327481748.0,7.003817e+08,123403419.0,4103.0,35000000.0,119.0,8.515187,8.845335,8.091327,7.544068,Horror,R,Warner Bros.
8,226277068.0,6.190214e+08,88411916.0,4071.0,97000000.0,304.0,8.354641,8.791706,7.946511,7.986772,"Action,Drama,Sci-Fi,Thriller",R,Twentieth Century Fox
9,264624300.0,1.034799e+09,72434025.0,4529.0,80000000.0,189.0,8.422630,9.014856,7.859943,7.903090,"Adventure,Animation,Comedy,Family,Fantasy,Sci-Fi",PG,Universal Pictures
10,115171585.0,1.405524e+08,31201920.0,2591.0,19000000.0,164.0,8.061345,8.147838,7.494181,7.278754,"Adventure,Comedy,Drama",R,Universal Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,169607287.0,2.359569e+08,515499.0,25.0,25000000.0,376.0,8.229445,8.372833,5.712228,7.397940,"Biography,Drama,History",PG,Twentieth Century Fox
2688,175003033.0,5.279659e+08,50198902.0,3773.0,NaN,280.0,8.243046,8.722606,7.700694,NaN,"Adventure,Animation,Comedy,Family,Fantasy",PG,Twentieth Century Fox
2689,176040665.0,2.554080e+08,33377060.0,2781.0,4500000.0,676.0,8.245613,8.407234,7.523448,6.653213,"Horror,Mystery,Thriller",R,Universal Pictures
2690,175750384.0,3.119504e+08,53003468.0,4088.0,80000000.0,325.0,8.244896,8.494086,7.724304,7.903090,"Action,Animation,Comedy,Family",PG,Warner Bros.


In [ ]:
plot_data = []

for genre in selected_genres:

    data_numericf = helper.filter_movies_by_genre(data_numeric, GENRES[genre])

    plot_data.append(
            dict(
            type="scatter",
            name=genre,
            x=(data_numericf['budget_log']),
            y=(data_numericf['opening_revenue_log']),
            color=GENRES_COLORS[genre],
        )
    )


data_numericf = helper.filter_movies_by_genre(data_numeric, GENRES['SciFi'])


data_numericf



In [70]:
distributors= data_numeric.groupby(['distributor']).sum()
distributors = distributors[distributors['budget']>0]

distributors['world_revenue_log']=np.log10(distributors['world_revenue'])
distributors['opening_revenue_log']=np.log10(distributors['opening_revenue'])
distributors['budget_log']=np.log10(distributors['budget'])
distributors['domestic_revenue_log']=np.log10(distributors['domestic_revenue'])
distributors['return'] = distributors['world_revenue']/distributors['budget']
distributors = distributors.sort_values('return', ascending=False)

# data = [
#         dict(
#             type = 'pie',
#             labels=distributors.index.tolist(), 
#             values=distributors['world_revenue_log'],
#             name = 'Pie Chart',
#             hole=0.5,
#             # showlegend=False,
#             )]
#     # layout_pie["title"] = 'Number of Groups per User'
#     # layout_pie['autosize']=False
# figure = dict(data=data, )
    

distributors


,domestic_revenue,world_revenue,opening_revenue,opening_theaters,budget,release_days,domestic_revenue_log,world_revenue_log,opening_revenue_log,budget_log,return
distributor,,,,,,,,,,,
A24,4.025057e+08,6.422683e+08,3.460902e+07,9546.0,4.100000e+06,10614.0,8.604772,8.807716,7.539189,6.612784,156.650801
Fox Searchlight Pictures,3.175226e+08,8.698939e+08,1.083615e+07,3290.0,6.000000e+06,4744.0,8.501775,8.939466,7.034875,6.778151,144.982316
United Artists Releasing,1.657247e+08,2.962010e+08,5.727317e+07,12932.0,1.000000e+07,926.0,8.219387,8.471586,7.757951,7.000000,29.620096
Roadside Attractions,1.919792e+08,2.737792e+08,2.915037e+07,6041.0,1.950000e+07,7124.0,8.283254,8.437400,7.464644,7.290035,14.039960
Focus Features,3.973403e+08,8.371920e+08,8.555083e+07,15228.0,6.000000e+07,6557.0,8.599163,8.922825,7.932224,7.778151,13.953201
Entertainment Studios Motion Pictures,6.919856e+07,9.199318e+07,4.949896e+06,5423.0,9.900000e+06,957.0,7.840097,7.963756,6.694596,6.995635,9.292241
Pure Flix Entertainment,4.200380e+07,4.922883e+07,1.406535e+07,4587.0,6.000000e+06,963.0,7.623289,7.692219,7.148151,6.778151,8.204805
The Weinstein Company,6.534160e+07,8.999617e+07,9.520929e+06,5510.0,1.100000e+07,752.0,7.815190,7.954224,6.978679,7.041393,8.181470
BH Tilt,3.730073e+07,5.000123e+07,1.536973e+07,5799.0,6.250000e+06,1263.0,7.571717,7.698981,7.186666,6.795880,8.000197
